In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# universal color map for Yeo's networks

In [ ]:
import numpy as np
import seaborn

In [ ]:
# load cmap17 values from txt file
def yeo_cmap(as_palette=False, networks=7):
    """load cmap7 or cmap17 from txt file in directory"""
    assert networks in (7, 17)
    cmap = {}
    with open('data/' + f'_cmap{networks}.txt', 'r') as f:
        for line in f.readlines():
            k, r, g, b = line.split()
            cmap[k] = (int(r), int(g), int(b))
    cmap = {k: np.array(v) / 255 for k, v in cmap.items()}

    if as_palette:  return seaborn.color_palette(cmap.values())
    else:   return cmap

In [ ]:
seaborn.set(style='white', context='notebook',rc={'figure.figsize': (7, 5)})

In [ ]:
cmap8 = yeo_cmap(as_palette=True)
cmap8

# gradients
large-scale manifold

In [ ]:
import pandas as pd

In [ ]:
approach = 'pca'

In [ ]:
RAND = 0    # random state

if you don't apply cosine kernel while making affinity mat, you get way skewed gradient values.

In [ ]:
# this is computed based on `baseline`
ref_cmat = pd.read_csv('data/cmat/ref_mean.csv').to_numpy()

In [ ]:
from brainspace.gradient import GradientMaps

In [ ]:
import seaborn
from matplotlib import pyplot as plt

In [ ]:
gref = GradientMaps(random_state=RAND, approach=approach, kernel='cosine')
# kernel needed for centered mats
gref.fit(ref_cmat)
seaborn.jointplot(data=pd.DataFrame(gref.gradients_), x=0, y=1)
plt.show()

In [ ]:
def cmat_cnt(subject, epoch): return pd.read_csv('data/cmat/cnt/cnt_'+str(subject)+'_'+epoch+'.csv').to_numpy()

In [ ]:
EPOCHS = ['baseline', 'early', 'late']
EPOCH_REF = 'baseline'

In [ ]:
subjects = pd.read_csv('data/subjects.csv')

In [ ]:
# g = GradientMaps(random_state=RAND, approach=approach, alignment='procrustes', kernel='cosine')
# g.fit([cmat_cnt(s, e) for e in EPOCHS for s in subjects.int_id], reference=gref.gradients_)
# np.stack(g.aligned_).shape

In [ ]:
# plt.hist(g.aligned_[0][:, 0], bins=50) # grad1
# plt.show()
# plt.hist(g.aligned_[0][:, 1], bins=50) # grad2
# plt.show()

# make dataframe

In [ ]:
# important: this is how the g.aligned_ list is organized
S = [1,2]
l = [(s, e) for e in EPOCHS for s in S]
print(l)
l[EPOCHS.index('late') * len(S) + S.index(2)]   # selects element for s=2, e='late'

In [ ]:
r = pd.read_csv('data/regions_sorted.csv')

In [ ]:
S = subjects.int_id.to_list()

# df = pd.concat([pd.DataFrame({'subject': s, 'epoch': e,
#                               'region': r['region'], '7net': r['7net'], '17net': r['17net'],
#                               'g1': g.aligned_[EPOCHS.index(e) * len(S) + S.index(s)][:, 0],
#                               'g2': g.aligned_[EPOCHS.index(e) * len(S) + S.index(s)][:, 1],
#                               'g3': g.aligned_[EPOCHS.index(e) * len(S) + S.index(s)][:, 2], })
#                 for e in EPOCHS for s in S], axis=0).reset_index(drop=True)

In [ ]:
# df

In [ ]:
# make use of palette

In [ ]:
names7nets = r['7net'].unique().tolist()

In [ ]:
# seaborn.displot(df.sample(10_000), x='g1', hue='7net', kind='kde', fill=True,
#                 palette=cmap8,
#                 hue_order=names7nets,
#                 )
# plt.show()
# # plt.savefig("sample10_000.svg", format="svg")

In [ ]:
# this is the 3d plot. it would mess up the notebook if run
# fig = px.scatter_3d(df.sample(1000), x='g1', y='g2', z='g3', color='label', opacity=.4)
# fig.update_traces(marker_size=3)
# fig.show()

## Eccentricity

the Euclidean distance from the manifold center. 3D space.
$Ecc = \sqrt{\sum_{\substack{i=1}}^3 G_i^2}$

In [ ]:
# df['ecc'] = df.apply(lambda r: np.sqrt(np.sum(g**2 for g in r[-3:])), axis=1)

In [ ]:
# df.to_csv('data/gradients.csv', index=False)

In [ ]:
df = pd.read_csv('data/gradients.csv')

In [ ]:
df.head()

average

In [ ]:
df_mean = df.groupby(['region', 'epoch', '7net', '17net']).agg('mean').drop('subject', axis=1).reset_index()
df_mean.head()

In [ ]:
# caution: `groupby` changes regions' ordering. This doesn't start with `LH_Vis`
df_mean = df_mean.set_index('region').loc[pd.Series(r['region'])].reset_index()
df_mean.head()

In [ ]:
g1range = (-5,7)
g2range = (-5,4)
g3range = (-4.5,4)

In [ ]:
seaborn.displot(df_mean[df_mean.epoch=='late'], x='g1', hue='7net', kind='kde', fill=True,
                palette=cmap8,hue_order=names7nets,
                )
plt.xlim(*g1range)
plt.savefig("g1 late.svg", format="svg")

# brain plot

## cortex

In [ ]:
import nibabel
from brainspace.datasets import load_conte69
from brainspace.utils.parcellation import map_to_labels
from surfplot import Plot

In [ ]:
fname = '/Users/qasem/PycharmProjects/gradients-rl-task/data/Schaefer2018_1000Parcels_7Networks_order.dlabel.nii'
l_hemi, r_hemi = load_conte69()
vertices = nibabel.load(fname).get_fdata()[0]

In [ ]:
data = df_mean[df_mean.epoch=='baseline'].reset_index()['g1']
data = map_to_labels(data, vertices, mask=(vertices != 0))
p = Plot(surf_lh=l_hemi, surf_rh=r_hemi, layout='row', size=(1600, 300))
p.add_layer(data, cbar=True, cmap='bwr', color_range=(-3.7, 3.7))
figure = p.build()
figure.savefig('test.png', dpi=300)

In [ ]:
df_mean.loc[df_mean.epoch=='baseline', 'g1'].hist(bins=100)

In [ ]:
data = df_mean[df_mean.epoch=='baseline'].reset_index()['g2']
data = map_to_labels(data, vertices, mask=(vertices != 0))
p = Plot(surf_lh=l_hemi, surf_rh=r_hemi, layout='row', size=(1600, 300))
p.add_layer(data, cbar=True, cmap='bwr', color_range=(-3.7, 3.7))
figure = p.build()
figure.savefig('test.png', dpi=300)

In [ ]:
data = df_mean[df_mean.epoch=='baseline'].reset_index()['g3']
data = map_to_labels(data, vertices, mask=(vertices != 0))
p = Plot(surf_lh=l_hemi, surf_rh=r_hemi, layout='row', size=(1600, 300))
p.add_layer(data, cbar=True, cmap='bwr', color_range=(-3.7, 3.7))
figure = p.build()
figure.savefig('test.png', dpi=300)

### eccentricity

In [ ]:
data = df_mean[df_mean.epoch=='baseline'].reset_index()['ecc']
data = map_to_labels(data, vertices, mask=(vertices != 0))
p = Plot(surf_lh=l_hemi, surf_rh=r_hemi, layout='row', size=(1600, 300))
p.add_layer(data, cbar=True,
            cmap='viridis',color_range=(1,5))
figure = p.build()
figure.savefig('test.png', dpi=300)

## subcortex
distribution of gradient values in Striatum

![ecc striatum](data/sample-subcortex.png)

PC 1

In [ ]:
df_mean.set_index(['epoch', 'region']).loc['baseline'][-14:]['g1'].hist()

PC 2

In [ ]:
df_mean.set_index(['epoch', 'region']).loc['baseline'][-14:]['g2'].hist()

PC 3

In [ ]:
df_mean.set_index(['epoch', 'region']).loc['baseline'][-14:]['g3'].hist()

### setup plotting

In [ ]:
subcortex_order = ['Left Accumbens',
                     'Left Amygdala',
                     'Left Caudate',
                     'Left Hippocampus',
                     'Left Pallidum',
                     'Left Putamen',
                     'Left Thalamus',
                     # 'Left Ventricles',
                     'Right Accumbens',
                     'Right Amygdala',
                     'Right Caudate',
                     'Right Hippocampus',
                     'Right Pallidum',
                     'Right Putamen',
                     'Right Thalamus',
                     # 'Right Ventricles',
                   ]

# to reorder data regarding function input
# array_name = data.loc[subcortex_order]
# two values removed, ventricles must set False

In [ ]:
data = df_mean.set_index(['epoch', 'region']).loc['baseline'].loc[subcortex_order]['g1']

In [ ]:
data

In [ ]:
from enigmatoolbox.plotting import plot_subcortical

plot_subcortical(data, ventricles=False,
                 size=(800, 400), cmap='bwr', color_bar=True, color_range=(-1, 1),
                 screenshot=True, filename='g1.png'
                 )

In [ ]:
data = df_mean.set_index(['epoch', 'region']).loc['baseline'].loc[subcortex_order]['g2']

plot_subcortical(data, ventricles=False,
                 size=(800, 400), cmap='bwr', color_bar=True, color_range=(-1, 1),
                 screenshot=True, filename='g2.png'
                 )

In [ ]:
data = df_mean.set_index(['epoch', 'region']).loc['baseline'].loc[subcortex_order]['g3']

plot_subcortical(data, ventricles=False,
                 size=(800, 400), cmap='bwr', color_bar=True, color_range=(-1, 1),
                 screenshot=True, filename='g3.png'
                 )

### eccentricity

In [ ]:
ecc_color_range = (1,6)    # (1, 1.5)

In [ ]:
data = df_mean.set_index(['epoch', 'region']).loc['baseline'].loc[subcortex_order]['ecc']

plot_subcortical(data, ventricles=False,
                 size=(800, 400), cmap='viridis', color_bar=True, color_range=ecc_color_range,
                 screenshot=True, filename='ecc.png'
                 )

# appendix scatter plots

In [ ]:
seaborn.jointplot(data=df_mean,
                  x='g1', y='g2',
                  xlim=g1range, ylim=g2range,
                  hue='7net',
                  palette=cmap8,hue_order=names7nets,
                  legend=False,
                       alpha=.5,
                  )
plt.savefig("g1g2.svg", format="svg")

In [ ]:
seaborn.jointplot(data=df_mean,
                  x='g1', y='g3',
                  xlim=g1range, ylim=g3range,
                  hue='7net',
                  palette=cmap8,hue_order=names7nets,
                  legend=False,
                       alpha=.5,
                  )
plt.savefig("g1g3.svg", format="svg")

In [ ]:
seaborn.jointplot(data=df_mean,
                  x='g3', y='g2',
                  xlim=g3range,
                  ylim=g2range,
                  hue='7net',
                  palette=cmap8,hue_order=names7nets,
                  legend=False, alpha=.5,
                  )
plt.savefig("g3g2.svg", format="svg")

# swarm plot eccentricity of one region for all subjects

In [ ]:
seaborn.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
seaborn.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
# set y axis range
seaborn.swarmplot(data=df[df.region=='7Networks_LH_Vis_1'], x='epoch', y='ecc')
plt.ylim(0, 6)
plt.show()

In [ ]:
regions = ['LH_DorsAttn_FEF_5', 'RH_DorsAttn_FEF_6',            # FEF
           'LH_Default_PFC_19', 'RH_Default_PFCdPFCm_3',        # PFC
           'LH_Default_pCunPCC_32', 'RH_Default_pCunPCC_18',    # PCC
           ]

seaborn.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
seaborn.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
fig, axes = plt.subplots(3, 2, figsize=(10, 10), sharex=True, sharey=True)

for i, region in enumerate(regions):
    seaborn.swarmplot(data=df[df.region=='7Networks_' + region], x='epoch', y='ecc',
                      ax=axes[i//2, i%2], color='black', alpha=.5)
    seaborn.pointplot(data=df[df.region=='7Networks_' + region], x='epoch', y='ecc', markers='X',
                      color='black', ci=None, join=True, scale=.3, linestyles='dashed', ax=axes[i//2, i%2])

    axes[i//2, i%2].set_ylim(0, 8)
    axes[i//2, i%2].set_title(region)

plt.tight_layout()
# plt.savefig('ecc-swarm-regions.svg', format='svg')
plt.show()

# variance cumulative explained

In [ ]:
g = GradientMaps(random_state=RAND, approach=approach, alignment='procrustes', kernel='cosine')
g.fit([cmat_cnt(s, e) for e in EPOCHS for s in subjects.int_id], reference=gref.gradients_)
np.stack(g.aligned_).shape

In [ ]:
len(g.lambdas_)

In [ ]:
e, s = 'baseline', 23
g.lambdas_[EPOCHS.index(e) * len(S) + S.index(s)]  # gives top 10 gradients

In [ ]:
df_var = pd.concat([pd.DataFrame({'subject': s, 'epoch': e,
                                  'eigenValue': g.lambdas_[EPOCHS.index(e) * len(S) + S.index(s)],
                                  'g': list(range(1,11)), })
                for e in EPOCHS for s in S], axis=0).reset_index(drop=True)
df_var.head()

In [ ]:
totalVariance = df_var.groupby(['subject', 'epoch'])['eigenValue'].sum().rename('totalVariance')

In [ ]:
totalVariance

In [ ]:
df_var.set_index(['subject', 'epoch'], inplace=True)

In [ ]:
df_var = df_var.join(totalVariance)

In [ ]:
df_var['varianceExplained'] = df_var['eigenValue'] / df_var['totalVariance']

In [ ]:
df_var.reset_index(inplace=True)

In [ ]:
df_var.head()

plot

In [ ]:
seaborn.set(rc={'figure.figsize': (7, 8)})

In [ ]:
# make percentage
df_var['varianceExplained'] = df_var['varianceExplained'] * 100

In [ ]:
y = 'varianceExplained'

df_var_mean = df_var.groupby(['epoch', 'g'])[y].mean()#.unstack(0)

cumulative = True
if cumulative:  df_var_mean = df_var_mean.groupby(level=0).apply(
    lambda x: pd.Series(x.to_numpy().cumsum(), index=list(range(1,11))).rename_axis('g'))

In [ ]:
df_var_mean.head()

In [ ]:
df_var_mean.groupby('g').mean()

In [ ]:
ax = seaborn.lineplot(x='g', y=y, data=df_var_mean.reset_index(), style='epoch', markers=True)
seaborn.move_legend(ax, "lower right", bbox_to_anchor=(1, 1))
plt.savefig("cumsum.svg", format="svg")